In [1]:
path = "/home/lunachocken/.steam/debian-installation/steamapps/libraryfolders.vdf"

In [2]:
import requests
from requests import Response
import json
import pandas as pd
import os
import re
import pandas as pd

In [3]:


# GET https://api.steampowered.com/ISteamApps/GetAppList/v2/
def fetch_app_list() -> Response:
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    print(response.status_code)
    return response

if os.path.exists('applist.json'):
    with open('applist.json', 'r') as f:
        response = json.load(f)
else:
    response = fetch_app_list()
    with open('applist.json', 'w') as f:
        json.dump(response.json(), f)   


In [4]:
# write json to file


    

In [5]:
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    data = response.json()

    if data[str(app_id)]['success']:
        game_data = data[str(app_id)]['data']
        return game_data
        
        # # print(game_data)
        # name = game_data['name']
        # description = game_data['short_description']
        
        # cloud = game_data['']
        
        # if 'price_overview' in game_data:
        #     price = game_data['price_overview']['final_formatted']
        # else:
        #     price = 'Free' if game_data['is_free'] else 'price not available'
        # return {
        #     'name': name,
        #     'description': description,
        #     'price': price
        # }
    else:
        return 'details not found'


def game_has_cloud(app_id):
    js = get_game_details(app_id)
    if js == 'details not found':
        # print("details not found")
        return False
    try:
        categories = {category['id']: category['description'] for category in js['categories']}
        categories[23]
        return True

    except KeyError:
        # print("Key Error")
        return False

In [6]:
# parse gameids from libraryfolders.vdf



with open(path, 'r') as f:
    data = f.read()
    matches = re.findall(r"[0-9]+\".+\"[0-9]+", data)
    print(matches)
    
    gameIDs= set([int(x.split('\"')[0]) for x in matches])
    
    

['440"\t\t"30301410253', '4000"\t\t"4231308880', '42910"\t\t"1804273306', '105600"\t\t"715107635', '228980"\t\t"346326452', '238370"\t\t"3515385168', '244850"\t\t"39642903247', '264710"\t\t"7060646067', '268650"\t\t"4661551670', '285920"\t\t"759164660', '286160"\t\t"3762437355', '361420"\t\t"3044500356', '427520"\t\t"2035193892', '431240"\t\t"5901017283', '526870"\t\t"27241375972', '544550"\t\t"3352808307', '548430"\t\t"3622974498', '553850"\t\t"96032784360', '585420"\t\t"6822884047', '602960"\t\t"1225468540', '647960"\t\t"321711902', '674940"\t\t"392892776', '690290"\t\t"396368443', '728880"\t\t"8707454190', '892970"\t\t"2340691335', '979120"\t\t"934513193', '1016920"\t\t"798929033', '1070560"\t\t"13004', '1127400"\t\t"104683623', '1161040"\t\t"10034804', '1236720"\t\t"720079502', '1329500"\t\t"1560050243', '1366540"\t\t"4562323525', '1391110"\t\t"649883581', '1422450"\t\t"27375076626', '1493710"\t\t"1339334355', '1628350"\t\t"778881638', '1966720"\t\t"1148646824', '2060160"\t\t"98247

In [10]:
print(gameIDs)

{105600, 427520, 1966720, 2060160, 2180100, 264710, 1391110, 431240, 275850, 1366540, 526870, 647960, 42910, 4000, 2732960, 238370, 544550, 892970, 2348590, 2768430, 728880, 979120, 1623730, 2767030, 976310, 440, 1628350, 2184130, 361420, 585420, 548430, 1493710, 286160, 602960, 1161040, 3490390, 1016920, 1329500, 285920, 1070560, 2211170, 2805730, 1127400, 268650, 1236720, 244850, 690290, 228980, 1422450, 553850, 674940}


In [11]:
# print first 3 keys from json

print(response['applist']['apps'][:3])

[{'appid': 1941401, 'name': ''}, {'appid': 1897482, 'name': ''}, {'appid': 2112761, 'name': ''}]


In [14]:
# Convert steam ids to game names using api results

gameNames = [(x['appid'], x['name']) for x in response['applist']['apps'] if x['appid'] in gameIDs]
# convert to dict
games = dict(gameNames)

print(games)

{431240: 'Golf With Your Friends', 427520: 'Factorio', 1422450: 'Deadlock', 1236720: 'Spirit Island', 2211170: 'Unrailed 2: Back on Track', 2184130: 'MareQuest', 286160: 'Tabletop Simulator', 268650: 'From The Depths', 2732960: 'Command & Conquer™ Generals Zero Hour', 1966720: 'Lethal Company', 892970: 'Valheim', 647960: 'Rusted Warfare - RTS', 602960: 'Barotrauma', 3490390: 'Sandustry Demo', 1623730: 'Palworld', 728880: 'Overcooked! 2', 544550: 'Stationeers', 2768430: 'ATLYSS', 2060160: 'The Farmer Was Replaced', 2767030: 'Marvel Rivals', 285920: 'TerraTech', 275850: "No Man's Sky", 553850: 'HELLDIVERS™ 2', 526870: 'Satisfactory', 548430: 'Deep Rock Galactic', 4000: "Garry's Mod", 105600: 'Terraria', 361420: 'ASTRONEER', 585420: 'Trailmakers'}


In [15]:
# has cloud
print(game_has_cloud(728880))

True


In [16]:
# Doesn't have cloud
print(game_has_cloud(553850))

False


In [17]:
import time, sys
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.6+
    count = len(it)
    start = time.time() # time estimate start
    def show(j):
        x = int(size*j/count)
        # time estimate calculation and string
        remaining = ((time.time() - start) / j) * (count - j)        
        mins, sec = divmod(remaining, 60) # limited to minutes
        time_str = f"{int(mins):02}:{sec:03.1f}"
        print(f"{prefix}[{u'█'*x}{('.'*(size-x))}] {j}/{count} Est wait {time_str}", end='\r', file=out, flush=True)
    show(0.1) # avoid div/0 
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

In [18]:
# Check for steam cloud support

games_results = {}

for app_id in progressbar(gameIDs):
    cloud = game_has_cloud(app_id)
    time.sleep(0.2)
    try:
        if cloud:
            games_results[app_id] = cloud
            # print(f"{games[app_id]}:{app_id} has cloud support")
        else:
            games_results[app_id] = cloud
            # print(f"{games[app_id]}:{app_id} has no cloud support")
    except KeyError:
        pass    
    

[████████████████████████████████████████████████████████████] 51/51 Est wait 00:0.01



In [19]:
games_results_l = sorted(games_results.items(), key=lambda x: x[1])

import colorama
from colorama import Fore, Back, Style

for item in games_results_l:
    try:
        print(f"{games[item[0]]}:{item[0]} {item[1]}")
    except KeyError:
        print(f"{item[0]}:{item[1]}")

Lethal Company:1966720 False
The Farmer Was Replaced:2060160 False
2180100:False
1391110:False
Golf With Your Friends:431240 False
1366540:False
42910:False
238370:False
2348590:False
ATLYSS:2768430 False
Marvel Rivals:2767030 False
440:False
1628350:False
MareQuest:2184130 False
Trailmakers:585420 False
1493710:False
1161040:False
Sandustry Demo:3490390 False
1016920:False
1329500:False
TerraTech:285920 False
1070560:False
2805730:False
From The Depths:268650 False
690290:False
228980:False
Deadlock:1422450 False
HELLDIVERS™ 2:553850 False
674940:False
Terraria:105600 True
Factorio:427520 True
264710:True
No Man's Sky:275850 True
Satisfactory:526870 True
Rusted Warfare - RTS:647960 True
Garry's Mod:4000 True
Command & Conquer™ Generals Zero Hour:2732960 True
Stationeers:544550 True
Valheim:892970 True
Overcooked! 2:728880 True
979120:True
Palworld:1623730 True
976310:True
ASTRONEER:361420 True
Deep Rock Galactic:548430 True
Tabletop Simulator:286160 True
Barotrauma:602960 True
Unraile

In [ ]:
# Tests

# import unittest

# class TestSteamMethods(unittest.TestCase):
#     def test_game_has_cloud(self):
#         self.assertEqual(game_has_cloud(728880), True)
#         self.assertEqual(game_has_cloud(553850), False)
        

# if __name__ == '__main__':
#     TestSteamMethods().test_game_has_cloud()

import pytest


